In [ ]:
import json
import math
import numpy as np

In [ ]:
with open('./data.json') as f:
    data = json.load(f)

In [ ]:
values = np.array(data['values'], dtype='float32')
values = values.reshape((data['height'], data['width']))[:10,]
values.shape

In [ ]:
import numpy as np
from bqplot import Figure, LinearScale, Axis, ColorScale
from bqplot_image_gl import ImageGL, Contour
import ipywidgets as widgets
scale_x = LinearScale(min=-1, max=4, allow_padding=False)
scale_y = LinearScale(min=-1, max=4, allow_padding=False)
scales = {'x': scale_x, 'y': scale_y}
axis_x = Axis(scale=scale_x, label='x')
axis_y = Axis(scale=scale_y, label='y', orientation='vertical')
scales_image = {'x': scale_x, 'y': scale_y, 'image': ColorScale(min=np.min(values).item(), max=np.max(values).item())}


In [ ]:
figure = Figure(scales=scales, axes=[axis_x, axis_y])
image = ImageGL(image=values, scales=scales_image, x=[0, 2], y=[0, 2])
figure.marks = (image, )

In [ ]:
figure

In [ ]:
from bqplot_image_gl.interacts import MouseInteraction, keyboard_events, mouse_events
from bqplot import PanZoom

In [ ]:
widget_label = widgets.Label(value="move cursor for information")
widget_label

In [ ]:
# if we want to work together with PanZoom, we don't want to processess drag events
panzoom = PanZoom(scales={'x': [scales_image['x']], 'y': [scales_image['y']]})
interaction = MouseInteraction(x_scale=scales_image['x'], y_scale=scales_image['y'], move_throttle=70, next=panzoom,
                               events=keyboard_events + mouse_events)
figure.interaction = interaction
def on_mouse_msg(interaction, data, buffers):
    # it might be a good idea to throttle on the Python side as well, for instance when many computations
    # happen, we can effectively ignore the queue of messages
    if data['event'] == 'mousemove':
        domain_x = data['domain']['x']
        domain_y = data['domain']['y']
        normalized_x = (domain_x - image.x[0]) / (image.x[1] - image.x[0])
        normalized_y = (domain_y - image.y[0]) / (image.y[1] - image.y[0])
        # TODO: think about +/-1 and pixel edges
        pixel_x = int(math.floor(normalized_x * image.image.shape[1]))
        pixel_y = int(math.floor(normalized_y * image.image.shape[0]))
        if pixel_x >= 0 and pixel_x < image.image.shape[1] and pixel_y >= 0 and pixel_y < image.image.shape[0]:
            value = str(image.image[pixel_y, pixel_x])
        else:
            value = "out of range"
        msg = f"x={pixel_x} y={pixel_y} value={value} (nx={normalized_x} ny={normalized_y})"
        widget_label.value = msg
    elif data['event'] == 'mouseleave':
        widget_label.value = "Bye!"
    elif data['event'] == 'mouseenter':
        widget_label.value = "Almost there..."  # this is is not visible because mousemove overwrites the msg
    else:
        widget_label.value = f'click {data}'
        
interaction.on_msg(on_mouse_msg)

In [ ]:
# cherry pick particular events:
# interaction.events = ['click']